In [127]:
import wrds
import pandas as pd

db = wrds.Connection()
db.create_pgpass_file()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [128]:
# Function to get features from WRDS given a list of tickers
def get_feature(ticker_list):
    start, end  = '1995-01-01', '2009-12-31'
    # helper: build a SQL-ready ('AAPL','MSFT',...) string -------
    tic_tuple = "(" + ",".join(f"'{t}'" for t in ticker_list) + ")"
    # ------------------------------------------------------------
    # 1.  Financial ratios  (monthly frequency, last-trading-day stamp)
    # ------------------------------------------------------------
    sql_ratios = f"""
        SELECT Public_date::date          AS date,
            ticker,
            pe_exi,
            de_ratio
        FROM   wrdsapps_finratio.firm_ratio
        WHERE  ticker IN {tic_tuple}
        AND  Public_date BETWEEN '{start}' AND '{end}'
    """
    ratios = db.raw_sql(sql_ratios)

    return ratios

In [129]:
energy_tickers = ['XOM', 'CVX', 'COP', 'BH', 'EOG', 'AESI', 'AROC', 'BTU', 'CLB', 'CNR', 'CRC', 'CRGY', 'CRK', 'CVI', 'DRQ']
energy_db = get_feature(energy_tickers)

materials_tickers = ['VMC', 'SHW', 'FCX', 'ECL', 'APD', 'AMR', 'ASIX', 'BCPC', 'CC', 'CE', 'CENX', 'ESI', 'FMC', 'FUL', 'HCC']
materials_db = get_feature(materials_tickers)

industrials_tickers = ['GE', 'CAT', 'UTX','RTX', 'UNP', 'HON', 'ABM', 'ACA', 'AIN', 'AIR', 'AL', 'ALG', 'ALGT', 'AMTM', 'AMWD', 'APOG'] # UTX is RTX
industrials_db = get_feature(industrials_tickers)

Consumer_Discretionary_tickers = ['AMZN', 'TSLA', 'HD', 'MCD', 'LOW', 'AAP', 'ABG', 'ADNT', 'AEO', 'ASO', 'ATGE', 'AXL', 'BJRI', 'BKE', 'BLMN']
Consumer_Discretionary_db = get_feature(Consumer_Discretionary_tickers)

Consumer_Staples_tickers = ['PG', 'COST', 'KO' ,'WMT', 'PEP', 'ANDE', 'BGS', 'CALM', 'CENT', 'CENTA', 'CHEF', 'ENR', 'EPC', 'FDP', 'FIZZ']
Consumer_Staples_db = get_feature(Consumer_Staples_tickers)

Healthcare_tickers = ['LLY', 'JNJ', 'ABBV', 'UNH', 'MRK', 'ACAD', 'ADMA', 'ADUS', 'AHCO', 'ALKS', 'AMN', 'AMPH', 'ANIP', 'AORT', 'ARWR']
Healthcare_db = get_feature(Healthcare_tickers)

Financials_tickers = ['BRK', 'JPM', 'V', 'B', 'MA', 'ABCB', 'ABR', 'ACT', 'AGO', 'ALRM', 'AMSF', 'APAM', 'AUB', 'AX', 'BANC'] # B is BAC
Financials_db = get_feature(Financials_tickers)

Information_Technology_tickers = ['ORCL', 'MSFT', 'NVDA', 'AAPL', 'AMD', 'ACIW', 'ACLS', 'ADEA', 'AEIS', 'AGYS', 'AOSL', 'ARLO', 'ATEN', 'BHE', 'BL'] 
Information_Technology_db = get_feature(Information_Technology_tickers)

Communication_Services_tickers =['FB','META', 'TMUS', 'GOOG', 'NFLX', 'DIS', 'ANGI', 'CABO', 'CARG', 'CARS', 'CCOI', 'CNK', 'GOGO', 'IAC', 'LUMN', 'MSGS'] # FB is META
Communication_Services_db = get_feature(Communication_Services_tickers)

Utilities_tickers = ['NEE', 'SO', 'D', 'DUK', 'AEP', 'AVA', 'AWR', 'CPK', 'CWEN', 'OTTR', 'CWT', 'MDU', 'MGEE', 'MSEX', 'NWN']
Utilities_db = get_feature(Utilities_tickers)

In [130]:
# for communication services, change the name of FB to META in TICKER column
Communication_Services_db['ticker'] = Communication_Services_db['ticker'].replace('FB', 'META')
# for financials, change the name of B to BAC in TICKER column
Financials_db['ticker'] = Financials_db['ticker'].replace('B', 'BAC')
# for finnncials, change the name of BRK to BRK.B in TICKER column
Financials_db['ticker'] = Financials_db['ticker'].replace('BRK', 'BRK-B')
# for industrials, change the name of UTX to RTX in TICKER column
industrials_db['ticker'] = industrials_db['ticker'].replace('UTX', 'RTX')

In [131]:
df = pd.concat([energy_db, materials_db, industrials_db, Consumer_Discretionary_db, Consumer_Staples_db, Healthcare_db, Financials_db, Information_Technology_db, Communication_Services_db, Utilities_db]) 

In [132]:
df

,date,ticker,pe_exi,de_ratio
0,1995-06-30,CRK,14.300847,2.877215
1,1995-07-31,CRK,16.419492,2.877215
2,1995-08-31,CRK,18.326271,2.877215
3,1995-09-30,CRK,13.216561,2.877215
4,1995-10-31,CRK,13.136943,2.877215
...,...,...,...,...
2077,2009-08-31,NWN,15.198556,2.527614
2078,2009-09-30,NWN,15.039711,2.527614
2079,2009-10-31,NWN,15.093863,2.527614
2080,2009-11-30,NWN,14.83737,2.561271


In [133]:
from curl_cffi import requests
import yfinance as yf

session = requests.Session(impersonate="chrome")
ticker = yf.Ticker('AAPL', session= session)

In [134]:
start_date = "1995-01-01"
end_date = "2009-12-31"

ticker.history(start=start_date, end=end_date, interval="1d")["Close"]

Date
1995-01-03 00:00:00-05:00    0.285078
1995-01-04 00:00:00-05:00    0.292507
1995-01-05 00:00:00-05:00    0.288792
1995-01-06 00:00:00-05:00    0.312007
1995-01-09 00:00:00-05:00    0.306087
                               ...   
2009-12-23 00:00:00-05:00    6.073949
2009-12-24 00:00:00-05:00    6.282526
2009-12-28 00:00:00-05:00    6.359766
2009-12-29 00:00:00-05:00    6.284330
2009-12-30 00:00:00-05:00    6.360668
Name: Close, Length: 3777, dtype: float64

In [135]:
# for each ticker, use y_finance to get the closed price of the stock from 2010-01-01 to 2024-12-31
# and add the closed price to the dataframe
#from curl_cffi import requests


import yfinance as yf

start_date = "1995-01-01"
end_date = "2009-12-31"

def get_monthly_stock_price(ticker, start_date, end_date):
    try:
        #session = requests.Session(impersonate="chrome")
        stock = yf.Ticker(ticker, session=session)
        daily_prices = stock.history(start=start_date, end=end_date, interval="1d")["Close"]
        month_end_prices = daily_prices.resample("ME").last()

        return month_end_prices
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

def get_daily_stock_price(ticker, start_date, end_date):
    try:
        #session = requests.Session(impersonate="chrome")
        stock = yf.Ticker(ticker,  session=session)
        daily_prices = stock.history(start=start_date, end=end_date, interval="1d")["Close"]
        return daily_prices
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None
    

In [136]:

# iterate through each ticker and get the stock price, create a new dataframe with each ticker with column "TICKER" storing the ticker name
tickers = df['ticker'].unique()
stock_prices = pd.DataFrame()
for ticker in tickers:
    month_end_prices = get_monthly_stock_price(ticker, start_date, end_date)

    if month_end_prices is not None:
        ticker_df = month_end_prices.reset_index()
        ticker_df.columns = ['date', 'close_price']
        ticker_df['ticker'] = ticker
        stock_prices = pd.concat([stock_prices, ticker_df], ignore_index=True)
    else:
        print(f"Skipping {ticker} due to error in fetching data.")
        print(ticker)

# change the date column to YY-MM-DD format
stock_prices['date'] = pd.to_datetime(stock_prices['date']).dt.strftime('%Y-%m-%d')
# merge the stock prices with the original dataframe on TICKER and public_date
df = pd.merge(df, stock_prices, on=['ticker', 'date'], how='left')


$BTU: possibly delisted; no price data found  (1d 1995-01-01 -> 2009-12-31) (Yahoo error = "Data doesn't exist for startDate = 788936400, endDate = 1262235600")


Error fetching data for BTU: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'
Skipping BTU due to error in fetching data.
BTU


$CRC: possibly delisted; no price data found  (1d 1995-01-01 -> 2009-12-31) (Yahoo error = "Data doesn't exist for startDate = 788936400, endDate = 1262235600")


Error fetching data for CRC: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'
Skipping CRC due to error in fetching data.
CRC


$DRQ: possibly delisted; no timezone found


...

Error fetching data for BL: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'
Skipping BL due to error in fetching data.
BL


$ATEN: possibly delisted; no price data found  (1d 1995-01-01 -> 2009-12-31) (Yahoo error = "Data doesn't exist for startDate = 788936400, endDate = 1262235600")


Error fetching data for ATEN: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'
Skipping ATEN due to error in fetching data.
ATEN


$META: possibly delisted; no price data found  (1d 1995-01-01 -> 2009-12-31) (Yahoo error = "Data doesn't exist for startDate = 788936400, endDate = 1262235600")


Error fetching data for META: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'
Skipping META due to error in fetching data.
META


In [137]:
df

,date,ticker,pe_exi,de_ratio,close_price
0,1995-06-30,CRK,14.300847,2.877215,17.650078
1,1995-07-31,CRK,16.419492,2.877215,20.731840
2,1995-08-31,CRK,18.326271,2.877215,21.011997
3,1995-09-30,CRK,13.216561,2.877215,19.611198
4,1995-10-31,CRK,13.136943,2.877215,19.611198
...,...,...,...,...,...
15061,2009-08-31,NWN,15.198556,2.527614,23.215185
15062,2009-09-30,NWN,15.039711,2.527614,22.972559
15063,2009-10-31,NWN,15.093863,2.527614,23.280903
15064,2009-11-30,NWN,14.83737,2.561271,23.876707


In [138]:
import pandas as pd
import numpy as np

# df has: date | ticker | pe | de_ratio | close_price  (monthly, end-of-month)
df = df.copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['ticker', 'date'])

# ---------------------------------------------------------------------
# 1. One-month simple return  r_t = P_t / P_{t-1} – 1
# ---------------------------------------------------------------------
df['ret'] = (
    df.groupby('ticker')['close_price']
      .transform(lambda s: s.pct_change())
)

# ---------------------------------------------------------------------
# 2. Expected return  – rolling 12-month mean of past simple returns, lagged 1 month
# ---------------------------------------------------------------------
df['exp_ret'] = (
    df.groupby('ticker')['ret']
      .transform(lambda s: s.rolling(window=12, min_periods=6)
                            .mean()
                            .shift(1))
)

# ---------------------------------------------------------------------
# 3. Volatility  – rolling 12-month std of past simple returns, annualised, lagged 1 month
# ---------------------------------------------------------------------
df['vol'] = (
    df.groupby('ticker')['ret']
      .transform(lambda s: s.rolling(window=12, min_periods=6)
                            .std()
                            .shift(1))
)

# ---------------------------------------------------------------------
# 4. Momentum  – “12-1” definition: cumulative return from t-13 to t-1
# ---------------------------------------------------------------------
df['mom'] = (
    df.groupby('ticker')['close_price']
      .transform(lambda s: (s.shift(1) / s.shift(13)) - 1)
)

# ---------------------------------------------------------------------
# 5. Tidy up
# ---------------------------------------------------------------------
df = df.drop(columns='ret')


/var/folders/2l/0y7rq65d4fzgrvt9vrsyq_580000gn/T/ipykernel_76185/523027035.py:14: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  .transform(lambda s: s.pct_change())


In [139]:
# Remove any row that contains at least one NaN in any column
df = df.dropna(how='any').reset_index(drop=True)
df

,date,ticker,pe_exi,de_ratio,close_price,exp_ret,vol,mom
0,2003-03-31,AAP,22.235577,3.196007,13.071248,-0.006888,0.104914,-0.129018
1,2003-04-30,AAP,23.913462,3.196007,14.057592,0.006321,0.118622,0.005435
2,2003-05-31,AAP,13.396396,3.412518,16.810337,-0.011304,0.083708,-0.159798
3,2003-06-30,AAP,13.716216,3.412518,17.211649,0.006985,0.102608,0.029067
4,2003-07-31,AAP,15.29955,3.412518,19.198484,0.013718,0.100665,0.117226
...,...,...,...,...,...,...,...,...
11157,2009-08-31,XOM,11.29902,1.031506,38.345531,-0.008158,0.048528,-0.105537
11158,2009-09-30,XOM,11.210784,1.031506,38.046097,-0.009114,0.048469,-0.115801
11159,2009-10-31,XOM,11.710784,1.031506,39.742950,-0.007318,0.048048,-0.096159
11160,2009-11-30,XOM,17.663529,1.042706,41.870628,0.000198,0.048568,-0.010755


In [140]:
import numpy as np

# read csv file in FF_month.csv, this is the monthly fama-french 5 factors
# the first column is date, the second column is Mkt-RF, the third column is SMB, the fourth column is HML, the fifth column is RMW, and the sixth column is CMA, the last column is RF
factor_df = pd.read_csv('FFmonth.csv')

# change the date column to YY-MM format
factor_df['date'] = pd.to_datetime(factor_df['date'], format='%Y%m').dt.strftime('%Y-%m')

# only keep the date that are in the range of 2000-01 to 2009-12
factor_df = factor_df[(factor_df['date'] >= '1995-01') & (factor_df['date'] <= '2009-12')]

# change the date column to YY-MM with the last day of the month
factor_df['date'] = pd.to_datetime(factor_df['date']).dt.to_period('M').dt.to_timestamp('M')

# merge the factor_df with the df on date
df = pd.merge(df, factor_df, on='date', how='left')


In [141]:
df['RF'] = df['RF'] / 100

In [142]:
df['ret_excess'] = df['exp_ret'] - df['RF']

In [143]:
df

,date,ticker,pe_exi,de_ratio,close_price,exp_ret,vol,mom,Mkt-RF,SMB,HML,RMW,CMA,RF,ret_excess
0,2003-03-31,AAP,22.235577,3.196007,13.071248,-0.006888,0.104914,-0.129018,1.09,0.66,-1.94,1.86,-0.78,0.0010,-0.007888
1,2003-04-30,AAP,23.913462,3.196007,14.057592,0.006321,0.118622,0.005435,8.22,1.01,1.15,-4.67,1.07,0.0010,0.005321
2,2003-05-31,AAP,13.396396,3.412518,16.810337,-0.011304,0.083708,-0.159798,6.05,4.82,0.39,-7.01,2.90,0.0009,-0.012204
3,2003-06-30,AAP,13.716216,3.412518,17.211649,0.006985,0.102608,0.029067,1.42,1.66,0.11,0.50,-0.39,0.0010,0.005985
4,2003-07-31,AAP,15.29955,3.412518,19.198484,0.013718,0.100665,0.117226,2.35,4.54,-1.24,-4.14,1.78,0.0007,0.013018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11157,2009-08-31,XOM,11.29902,1.031506,38.345531,-0.008158,0.048528,-0.105537,3.33,-0.09,7.63,-3.03,3.34,0.0001,-0.008258
11158,2009-09-30,XOM,11.210784,1.031506,38.046097,-0.009114,0.048469,-0.115801,4.08,2.73,1.04,1.31,0.36,0.0001,-0.009214
11159,2009-10-31,XOM,11.710784,1.031506,39.742950,-0.007318,0.048048,-0.096159,-2.59,-4.94,-4.21,4.17,-1.50,0.0000,-0.007318
11160,2009-11-30,XOM,17.663529,1.042706,41.870628,0.000198,0.048568,-0.010755,5.56,-2.68,-0.34,1.00,0.13,0.0000,0.000198


In [144]:
df.to_csv('feature_data/crisis_data.csv')

## Daily Data

In [ ]:
# iterate through each ticker and get the stock price, create a new dataframe with each ticker with column "TICKER" storing the ticker name
tickers = df['ticker'].unique()
stock_daily_prices = pd.DataFrame()
for ticker in tickers:
    daily_price = get_daily_stock_price(ticker, start_date, end_date)
    #print(len(month_end_prices))
    #if len(month_end_prices)!= 120:
        #print(f"Warning: {ticker} does not have 120 months of data. Found {len(month_end_prices)} months.")
    if daily_price is not None:
        ticker_df = daily_price.reset_index()
        ticker_df.columns = ['date', 'close_price']
        ticker_df['ticker'] = ticker
        stock_daily_prices = pd.concat([stock_daily_prices, ticker_df], ignore_index=True)
    else:
        print(f"Skipping {ticker} due to error in fetching data.")
        print(ticker)

$DRQ: possibly delisted; no timezone found
C:\Users\ksg\AppData\Local\Temp\ipykernel_51656\3801793463.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stock_daily_prices = pd.concat([stock_daily_prices, ticker_df], ignore_index=True)


In [ ]:
price_df = stock_daily_prices

In [ ]:
import numpy as np
# assume price_df has columns ['TICKER', 'Date', 'Close']
price_df['date'] = pd.to_datetime(price_df['date'])
price_wide = (price_df
              .pivot(index='date', columns='ticker', values='close_price')
              .sort_index())

# month-end prices
p_m = price_wide.resample('ME').last()

# one-period **log** return: r_t = ln(P_t / P_{t-1})
r_m = np.log(p_m / p_m.shift(1))

In [ ]:
exp_ret_12m = r_m.rolling(window=12, min_periods=6).mean().shift(1)


In [ ]:
vol_12m = r_m.rolling(window=12, min_periods=6).std().shift(1) * np.sqrt(12)


In [ ]:
stock_daily_prices = stock_daily_prices.sort_values(['ticker', 'date'])
# 3. Calculate daily returns per ticker
stock_daily_prices['daily_return'] = stock_daily_prices.groupby('ticker')['close_price'].pct_change()\


monthly = (
    stock_daily_prices.set_index('date')
        .groupby('ticker')
        .resample('ME')
        .agg(
            price_end=('close_price', 'last'),
            daily_returns=('daily_return', list)
        )
)
# 5. Compute expected_return and volatility from daily_returns list
monthly['expected_return'] = monthly['daily_returns'].apply(lambda x: pd.Series(x).mean())
monthly['volatility'] = monthly['daily_returns'].apply(lambda x: pd.Series(x).std())

# Drop the intermediate daily_returns column
monthly = monthly.drop(columns='daily_returns')


# 6. Compute momentum: cumulative return over the past `momentum_lookback` months
def add_momentum(group: pd.DataFrame) -> pd.DataFrame:
    momentum_lookback = 12
    group = group.copy()
    group['momentum'] = group['price_end'] / group['price_end'].shift(momentum_lookback) - 1
    return group

monthly = monthly.groupby('ticker').apply(add_momentum)

monthly.index = monthly.index.droplevel(0)  # Remove the ticker level from the index
monthly.reset_index(inplace=True)
# change the public_date column to YY-MM-DD format
monthly['date'] = pd.to_datetime(monthly['date']).dt.strftime('%Y-%m-%d')

# drop the column "price_end"
monthly.drop(columns='price_end', inplace=True)

In [ ]:
monthly

,ticker,date,expected_return,volatility,momentum
0,AAP,2015-01-31,0.000255,0.015170,NaN
1,AAP,2015-02-28,-0.001181,0.019376,NaN
2,AAP,2015-03-31,-0.001474,0.012157,NaN
3,AAP,2015-04-30,-0.002068,0.014913,NaN
4,AAP,2015-05-31,0.003571,0.015585,NaN
...,...,...,...,...,...
16095,XOM,2024-08-31,0.000198,0.013174,0.097898
16096,XOM,2024-09-30,-0.000176,0.016501,0.031893
16097,XOM,2024-10-31,-0.000094,0.012030,0.141942
16098,XOM,2024-11-30,0.000985,0.012420,0.187306


In [ ]:
# merge the monthly dataframe with the original dataframe on TICKER and date
df = pd.merge(df, monthly, on=['ticker', 'date'], how='left')
df

,Unnamed: 0,date,ticker,pe_exi,de_ratio,close_price,expected_return,volatility,momentum
0,0,2015-01-31,EOG,15.983842,0.981159,66.421738,-0.001389,0.027779,NaN
1,1,2015-02-28,EOG,16.864662,0.962599,66.936546,0.000599,0.020250,NaN
2,2,2015-03-31,EOG,17.234962,0.962599,68.406288,0.001111,0.016134,NaN
3,3,2015-04-30,EOG,18.599624,0.962599,73.953873,0.003793,0.012399,NaN
4,4,2015-05-31,EOG,23.339474,0.971782,66.285713,-0.005347,0.015231,NaN
...,...,...,...,...,...,...,...,...,...
15334,15334,2024-08-31,MSEX,30.587379,1.923909,61.879597,-0.002015,0.019154,-0.144272
15335,15335,2024-09-30,MSEX,31.669903,1.923909,64.069588,0.001933,0.020150,0.006511
15336,15336,2024-10-31,MSEX,29.703883,1.923909,60.092247,-0.002676,0.014918,-0.015553
15337,15337,2024-11-30,MSEX,28.450000,1.878579,64.598839,0.003882,0.023564,0.047005


In [ ]:
# reset stock_daily_prices index
stock_daily_prices.reset_index(drop=True, inplace=True)

# change the public_date column to YY-MM-DD format
stock_daily_prices['public_date'] = pd.to_datetime(stock_daily_prices['public_date']).dt.strftime('%Y-%m-%d')

# change the name public_date to date
stock_daily_prices.rename(columns={'public_date': 'date'}, inplace=True)

stock_daily_prices

In [ ]:
daily_factor_df = pd.read_csv('FFdaily.csv')

# change the date column to YY-MM-DD format
daily_factor_df['date'] = pd.to_datetime(daily_factor_df['date'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

# merge the daily_factor_df with the stock_daily_prices on date
daily_factor_df = pd.merge(stock_daily_prices, daily_factor_df, on='date', how='left')

daily_factor_df

In [ ]:
# ------------------------------------------------------------
# 0.  Basic housekeeping
# ------------------------------------------------------------
daily_factor_df = daily_factor_df.sort_values(['TICKER', 'date'])

# ------------------------------------------------------------
# 1.  Compute *asset* returns  (and optionally excess returns)
# ------------------------------------------------------------
daily_factor_df['ret'] = daily_factor_df.groupby('TICKER')['close_price'].pct_change()

# excess return is often what you model in a FF-style regression
daily_factor_df['ret_excess'] = daily_factor_df['ret'] - daily_factor_df['RF']

In [ ]:
# create the ticker list 
ticker_list = daily_factor_df['TICKER'].unique().tolist()

In [ ]:
# convert the date column to datetime format
daily_factor_df['date'] = pd.to_datetime(daily_factor_df['date'])
# only keep date that are after 2016-01-01
daily_factor_df = daily_factor_df[daily_factor_df['date'] >= '2016-01-01']
daily_factor_df.reset_index(drop=True, inplace=True)